In [1]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# This model will use tensorflows functional API which is better for hard core deep learning models as 
# opposed to TF's sequential API
# The model we are building is called a siamese neural network, which allows us to do one shot classification.
# https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

# Import standard tensorflow dependencies - Funtional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [3]:
# Only use if training o a GPU based machine. This notebook will be trained using google co lab
# Avoid out of memory errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
gpus

[]

In [5]:
# Setup paths
POS_PATH = os.path.join('../raw_data', 'positve')
NEG_PATH = os.path.join('../raw_data', 'negative')
ANC_PATH = os.path.join('../raw_data', 'anchor')

In [ ]:
DATA_PATH = os.path.join('../raw_data', 'data')

In [ ]:
# Make directories
os.mkdir(POS_PATH)
os.mkdir(NEG_PATH)
os.mkdir(ANC_PATH)

In [ ]:
os.mkdir(DATA_PATH)

In [ ]:
# Downloaded images as zipped tar file from http://vis-www.cs.umass.edu/lfw/
# Command below unzips tar file and saves in data directory
!tar -xf ../raw_data/lfw.tgz -C ../raw_data/data

In [ ]:
# Move LFW images to the following repository ../raw_data/negative

for directory in os.listdir('../raw_data/data/lfw'):
    for file in os.listdir(f'../raw_data/data/lfw/{directory}'):
        EX_PATH = os.path.join('../raw_data/data/lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [7]:
import uuid

In [10]:
# Establish connection to webcam

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # cut down frame to 250x250px
    frame = frame[270:270+250,650:650+250, :]
    
    # Collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
        #cv2.waitKey(1)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    # show image back to screen
    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

# Release the webcam
cap.release()

# Close the image show frame
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [ ]:
frame.shape


In [ ]:
plt.imshow(frame[270:270+250,650:650+250, :])

In [ ]:
plt.imshow(frame[120:120+250,200:200+250, :])